In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import truncnorm

In [2]:
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/wheat-seeds.csv"
df = pd.read_csv(url, header=None)
df.columns = ['area', 
              'perimeter',
              'compactness',
              'length of kernel',
              'width of kernel',
              'asymmetry coefficient',
              'length of kernel groove',
             'target']

In [3]:
df

,area,perimeter,compactness,length of kernel,width of kernel,asymmetry coefficient,length of kernel groove,target
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
...,...,...,...,...,...,...,...,...
205,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,3
206,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3
208,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,3


In [4]:
y = df.iloc[:, -1]

In [5]:
X = df.drop('target', axis=1)

array([0. , 0.5, 1. ])

In [18]:
X = pd.DataFrame(np.linspace(0, 1, num=3).reshape(-1, 1), columns=['Dosage'])
X

,Dosage
0,0.0
1,0.5
2,1.0


In [20]:
target = np.array([0, 1, 0]).reshape(-1, 1)
target

array([[0],
       [1],
       [0]])

# 1. Initialize Network

In [211]:
def init_weights(X, n_neurons, random_state=42):
    n_inputs = int(X.shape[1])
    stddev = 2 / np.sqrt(n_inputs + n_neurons)
    b = np.zeros(n_neurons)
    weights = pd.DataFrame(truncnorm.rvs(-1, 1, size=(n_inputs, n_neurons), scale=stddev, random_state=random_state),
                           columns = [str(i) for i in range(1, n_neurons + 1)],
                           index=['w' + str(x) for x in range(1, n_inputs + 1)])
    
    return pd.DataFrame(weights), b

In [212]:
weights, biases = init_weights(X, 2)
display(weights)
biases

,1,2
w1,-0.249842,1.003948


array([0., 0.])

# 2. Forward Propagate

In [62]:
def softmax(x):
    return np.log(1 + np.exp(x))

In [63]:
def crossenthropy(x):
    return 1. / (1. + np.exp(-x))

In [86]:
def neuron_layer(X, weights, b, activation=None):
    Z = np.dot(X, weights) + b
    if activation is not None:
        Z = activation(Z)
        pass
    Z = pd.DataFrame(Z,
                     columns=['n' + str(i) for i in range(1, weights.shape[1] + 1)],
                     index = [str(i) + ' object' for i in range(1, X.shape[0] + 1)])
    return Z

In [87]:
hidden1 = neuron_layer(X, weights, biases, activation=None)
hidden1

,n1,n2
1 object,0.000000,0.000000
2 object,-0.124921,0.501974
3 object,-0.249842,1.003948


In [89]:
weights_2, biases_2 = init_weights(hidden1, 1, random_state=43)
display(weights_2)
biases_2

,1
w1,-0.825995
w2,0.216780


array([0.])

In [90]:
output = neuron_layer(hidden1, weights_2, biases_2, activation=None)
output

,n1
1 object,0.000000
2 object,0.212002
3 object,0.424004


# 3. Back Propagate Error

## Sum of squared residuals

$$\Large SSR = \sum_{i-1}^n{(Observed_i - Predicted_i)^2}$$

In [427]:
def ssr(target, predicted):
#     return (-1.0 / len(target)) * np.sum(target * np.log(predicted) + (1-target) * np.log(1 - predicted))
    return np.sum((target - predicted) ** 2)

In [358]:
ssr(target, output)

n1    0.80072
dtype: float64

## Derivative of b3

$$\Large \frac{dSSR}{db_3} = \frac{dSSR}{d Predicted} \cdot \frac{d Predicted}{d b_3}$$


$$\Large \frac{dSSR}{db_3} = \sum_{i=1}^n{-2 \cdot (Observed_i - Predicted_i)} \cdot 1$$

In [114]:
def derivative_of_b3(target, output):
    return np.sum((-2) * (target - output))

In [115]:
derivative_of_b3(target, output)

n1   -0.727988
dtype: float64

In [117]:
biases_2

array([0.])

In [119]:
learning_rate = 0.1
gradient = derivative_of_b3(target, output)
step_size_b3 = gradient * learning_rate
biases_3 = biases_2 - step_size_b3
biases_3

n1    0.072799
dtype: float64

## Derivative of w3 and w4

$$\Large \frac{dSSR}{dw_3} = \frac{dSSR}{dPredicted} \cdot \frac{dPredicted}{dw_3}$$

$$\Large \frac{dSSR}{dw_4} = \frac{dSSR}{dPredicted} \cdot \frac{dPredicted}{dw_4}$$

$$\Large \frac{dSSR}{dPredicted} = \sum_{i=1}^n{-2 \cdot (Observed_i - Predicted_i)}$$

$$\Large \frac{dPredicted}{dw_3} = \frac{d}{dw_3}(y_{1,i}w_3 + y_{2,i}w_4 + b_3) = y_{1,i}$$


$$\Large \frac{dPredicted}{dw_4} = \frac{d}{dw_4}(y_{1,i}w_3 + y_{2,i}w_4 + b_3) = y_{2,i}$$

In [176]:
def derivative_weights(target, output, hidden1):
    derivative = []
    for i in range(1, hidden1.shape[1] + 1):
        derivative.append(np.dot((-2.) * (target - output).T, hidden1['n' + str(i)]))
    return np.array(derivative).reshape(-1, output.shape[1])

In [177]:
gradient_weights = derivative_weights(target, output, hidden1)
gradient_weights

array([[-0.01499303],
       [ 0.06024695]])

In [178]:
step_size_weights = gradient_weights * 0.1
step_size_weights

array([[-0.0014993 ],
       [ 0.00602469]])

In [179]:
new_weights_2 = weights_2 - step_size_weights
new_weights_2

,1
w1,-0.824496
w2,0.210755


In [201]:
display(weights_2)

,1
w1,-0.825995
w2,0.216780


## Derivative of w1 and w2 and b1,b2

$$\Large \frac{dSSR}{dw_1} = \frac{dSSR}{dPredicted} \cdot \frac{dPredicted}{dy_1} \cdot \frac{dy_1}{dx_1} \cdot \frac{dx_1}{dw_1}$$

$$\Large \frac{dSSR}{dw_2} = \frac{dSSR}{dPredicted} \cdot \frac{dPredicted}{dy_2} \cdot \frac{dy_2}{dx_2} \cdot \frac{dx_2}{dw_2}$$

$$\Large \frac{dSSR}{dPredicted} = \sum_{i=1}^n{-2 \cdot (Observed_i - Predicted_i)}$$

$$\Large \frac{dPredicted}{dy_1} = \frac{d}{dy_1}(y_{1,i}w_3 + y_{2,i}w_4 + b_3) = w_3$$


$$\Large \frac{dy_1}{dx_1} = \frac{d}{dx_1}ln(1 + e^x) = \frac{e^x}{e^x + 1}$$

$$\Large \frac{dx_1}{dw_1} = \frac{d}{dw_1}(Input_i \cdot w_1 + b_1) = Input_i$$

In [353]:
def derivative_hidden(target, output, weights_2, X, n_neurons):
    gradients_w = []
    gradients_b = []
    for i in range(n_neurons):
        dSSR_dPred = np.sum((-2.) * (target - output))
        dPred_dy = weights_2.iloc[i, 0]
        dy_dx = np.array(list(map(lambda x: np.exp(x) / (1 + np.exp(x)), X.T))).reshape(X.shape[1], X.shape[0])
        dx_dw = X
        dSSR_dw = np.dot(np.dot(np.dot(dSSR_dPred, dPred_dy), dy_dx), dx_dw)
        
        dSSR_db = np.dot(np.dot(np.dot(dSSR_dPred, dPred_dy), dy_dx), np.ones(X.shape))
        
        gradients_w.append(dSSR_dw)
        
        gradients_b.append(dSSR_db)
    return np.array(gradients_w), np.array(gradients_b)

In [354]:
derivative_hidden(target, output, weights_2, X, 2)

(array([[ 0.74943387],
        [-0.19668678]]),
 array([[ 1.26988901],
        [-0.33327875]]))

In [400]:
def derivative_hidden_test(target, output, weights_2, X, n_neurons):
    gradients_w = []
    gradients_b = []
    for i in range(n_neurons):
        summa_w = 0.0
        summa_b = 0.0
        for j in range(len(X)):
            summa_w += (-2.) * (target[j] - output.iloc[j, 0]) * weights_2.iloc[i, 0] * (np.exp(X.iloc[j, 0]) / (1 + np.exp(X.iloc[j, 0]))) * X.iloc[j, 0]
            summa_b += (-2.) * (target[j] - output.iloc[j, 0]) * weights_2.iloc[i, 0] * (np.exp(X.iloc[j, 0]) / (1 + np.exp(X.iloc[j, 0])))
        gradients_w.append(summa_w)
        gradients_b.append(summa_b)
    return np.array(gradients_w), np.array(gradients_b)

In [401]:
derivative_hidden_test(target, output, weights_2, X, 2)

(array([[-0.10692243],
        [ 0.02806149]]),
 array([[ 0.29822533],
        [-0.07826839]]))

In [314]:
# summa = 0.
# for i in range(len(X)):
#     summa += (-2.) * (target[i] - output.iloc[i, 0]) * weights_2.iloc[0, 0] * (np.exp(X.iloc[i, 0]) / (1 + np.exp(X.iloc[i, 0]))) * X.iloc[i, 0]
# summa

array([-0.10692243])

In [428]:
epochs = 10
def train(X, target, epochs=10, learning_rate = 0.1):
    
    weights_1, biases_1 = init_weights(X, 2)
    
    hidden1 = neuron_layer(X, weights_1, biases_1, activation=None)
    
    weights_2, biases_2 = init_weights(hidden1, 1, random_state=43)
    output = neuron_layer(hidden1, weights_2, biases_2, activation=None)
    for epoch in range(epochs):
        error = ssr(target, output)
        print(epoch, ": Error:", np.array(error))
        
        ## b3
        gradient_b3 = derivative_of_b3(target, output)
        step_size_b3 = gradient * learning_rate
        
        print(biases_2)
        biases_2 = np.array(biases_2 - step_size_b3)
        
        ##-- w3, w4
        
        display(weights_2)
        
        gradient_weights = derivative_weights(target, output, hidden1)
        step_size_weights = gradient_weights * learning_rate
        weights_2 = weights_2 - step_size_weights
        
        ##-- w1, w2, b1, b2
        print(weights_1)
        print(biases_1)
        
        gradient_weights_1_2, gradient_biases_1_2 = derivative_hidden_test(target, output, weights_2, X, 2)
        step_size_w_1_2 = gradient_weights_1_2 * learning_rate
        step_size_b_1_2 = gradient_biases_1_2 * learning_rate
        weights_1 = weights_1 - step_size_w_1_2.T
        biases_1 = biases_1 - step_size_b_1_2.T
        
        
        hidden1 = neuron_layer(X, weights_1, biases_1, activation=crossenthropy)
        output = neuron_layer(hidden1, weights_2, biases_2, activation=crossenthropy)
    
    return output

In [429]:
train(X, target, epochs=10, learning_rate=0.1)

0 : Error: [0.80072024]
[0.]


,1
w1,-0.825995
w2,0.216780


           1         2
w1 -0.249842  1.003948
[0. 0.]
1 : Error: [0.71130634]
[0.0727988]


,1
w1,-0.824496
w2,0.210755


           1        2
w1 -0.239169  1.00122
[[-0.0297684   0.00760932]]
2 : Error: [0.71457612]
[0.14559759]


...

,1
w1,-1.103745
w2,-0.123684


        1         2
w1  0.003  0.995875
[[0.29842259 0.00108662]]
9 : Error: [0.72045393]
[0.65518916]


,1
w1,-1.150172
w2,-0.172522


           1         2
w1  0.043013  1.001877
[[0.35460095 0.00951317]]


,n1
1 object,0.474006
2 object,0.464244
3 object,0.455342


In [378]:
target

array([[0],
       [1],
       [0]])

In [408]:
from math import exp
from random import seed
from random import random
 
# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network
 
# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation
 
# Transfer neuron activation
def transfer(activation):
	return 1.0 / (1.0 + exp(-activation))
 
# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs
 
# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)
 
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
 
# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] += l_rate * neuron['delta']
 
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)
		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
 
# Test training backprop algorithm
seed(1)
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 20, n_outputs)
for layer in network:
	print(layer)

>epoch=0, lrate=0.500, error=6.350
>epoch=1, lrate=0.500, error=5.531
>epoch=2, lrate=0.500, error=5.221
>epoch=3, lrate=0.500, error=4.951
>epoch=4, lrate=0.500, error=4.519
>epoch=5, lrate=0.500, error=4.173
>epoch=6, lrate=0.500, error=3.835
>epoch=7, lrate=0.500, error=3.506
>epoch=8, lrate=0.500, error=3.192
>epoch=9, lrate=0.500, error=2.898
>epoch=10, lrate=0.500, error=2.626
>epoch=11, lrate=0.500, error=2.377
>epoch=12, lrate=0.500, error=2.153
>epoch=13, lrate=0.500, error=1.953
>epoch=14, lrate=0.500, error=1.774
>epoch=15, lrate=0.500, error=1.614
>epoch=16, lrate=0.500, error=1.472
>epoch=17, lrate=0.500, error=1.346
>epoch=18, lrate=0.500, error=1.233
>epoch=19, lrate=0.500, error=1.132
[{'weights': [-1.4688375095432327, 1.850887325439514, 1.0858178629550297], 'output': 0.029980305604426185, 'delta': -0.0059546604162323625}, {'weights': [0.37711098142462157, -0.0625909894552989, 0.2765123702642716], 'output': 0.9456229000211323, 'delta': 0.0026279652850863837}]
[{'weights